## Dependencies

In [ ]:
%pip install generative-ai-hub-sdk[all]
%pip install hdbcli

## Get Embeddings

In [ ]:
from gen_ai_hub.proxy.native.openai import embeddings
import json, struct

# Required if you want to insert the data into a HANA Database
def createFvecs(values) -> bytes:
    return struct.pack("<I%sf" % len(values), len(values), *values)

# Get the embeddings from the AI Core
# Model can be changed to whatever you are using
# Check the README on how to setup auth
def get_embedding(input: list[str], model="text-embedding-3-small") -> list[bytes]:
    response = embeddings.create(
      model_name=model,
      input=input
    )
    return list(map(lambda data: createFvecs(data.embedding), response.data))

vectors: list[bytes]

with open("./data.json") as file:
    data = json.load(file)
    materials = data["value"]
    request = list(map(lambda material: f"""
            MaterialNumber:{material["material_number"]},
            Description{material["description"]},
        """, materials))
    vectors = get_embedding(request)

# Insert data into HANA Database

In [ ]:
from dotenv import load_dotenv
from hdbcli import dbapi
import os

# Get the credentials stored in a local .env
# You could also get them from a destination or however you like
def connect_to_hana():
    load_dotenv()
    return dbapi.connect(
        address=os.getenv('host'),
        port=os.getenv('port'),
        user=os.getenv('user'),
        password=os.getenv('password'),
        encrypt=True
    )

conn = connect_to_hana()
cursor = conn.cursor()

table_name = "YOUR_TABLE_NAME"
column_name = "COLUMN_NAME_OF_YOUR_VECTORS"

# The `?` is a prepared statement and will be replace by your vectors in the next line of code
sql_query = f"INSERT INTO {table_name} ({column_name}) VALUES (?)"
cursor.executemany(sql_query, vectors)

cursor.close()